In [ ]:
import numpy as np
#from dataclasses import dataclass, field
#from typing import List, Dict, Optional, Tuple

: 

In [2]:
from uavmec.system import Vehicle, RSU, UAV, UavVecWorld

In [ ]:
# ============================================
# Example world construction & one step (MULTI-UAV)
# ============================================
"""
        action format :

        {
          rsu_id: {
              uav_id: offload_ratio,
              ...
          },
          ...
        }
        """
# ----- Example parameters -----
M = 10   # vehicles
R = 4    # RSUs
U = 2    # UAVs

# Vehicles
vehicles = []
for vid in range(M):
    lam = 0.5
    c_task = 5e8
    d_task = 5e5
    rsu_id = np.random.randint(0, R)
    gpu_frac = 0.3 if (vid % 3 == 0) else 0.0
    vehicles.append(Vehicle(vid, lam, c_task, d_task, rsu_id, gpu_frac))

# RSUs
rsus = []
for rid in range(R):
    x = (rid % 2) * 500.0
    y = (rid // 2) * 500.0
    f_max = 8e9
    bandwidth = 20e6
    noise_power = 1e-13
    tx_power = 1.0

    gpu_flops = 1e12 if rid % 2 == 0 else 0.0
    gpu_memory = 4096 if gpu_flops > 0 else 0.0
    gpu_power_active = 120.0 if gpu_flops > 0 else 0.0

    rsus.append(
        RSU(
            rid, x, y,
            f_max, bandwidth, noise_power, tx_power,
            gpu_flops, gpu_memory, gpu_power_active
        )
    )

# UAVs (MULTIPLE)
uavs = []
for uid in range(U):
    uavs.append(
        UAV(
            uid=uid,
            x=0.0,
            y=0.0,
            H=30.0,
            f_u=5e9,
            hover_power=220.0,
            energy_coeff=1e-28,
            fly_coeff=1.0,
            gpu_flops=2e12,
            gpu_memory=8192,
            gpu_power_active=200.0,
            E_max=5e5,
            E_batt=5e5,
            EH_max=1e3,
            mu1=15.0,
            mu2=0.5,
            g0=1.0,
            zeta=0.2,
            H0=0.0
        )
    )

# World
world = UavVecWorld(
    vehicles=vehicles,
    rsus=rsus,
    uavs=uavs,                     # <<< CHANGED
    delta_t=1.0,
    rsu_capacity_threshold=4e8,
    uplink_power_ue=0.5,
    uplink_noise_ue=1e-13,
    rsu_radius=300.0
)

world.reset()

# ----- Example cooperative action -----
# Two UAVs cooperate to serve RSU 0
dummy_action = {
    0: {        # RSU 0
        0: 0.3, # UAV 0 offloads 30%
        1: 0.4  # UAV 1 offloads 40%
    }
}

info = world.step(dummy_action)
info


{'time_slot': 1,
 'veh_stats': {'veh_tasks': {0: 0,
   1: 2,
   2: 1,
   3: 0,
   4: 1,
   5: 1,
   6: 2,
   7: 1,
   8: 1,
   9: 1},
  'rsu_upload_delay': {0: 0.0005926267948526406,
   1: 0.0011852535897052812,
   2: 0.00948202871764225,
   3: 0.0071115215382316875}},
 'uav_result': {'served_rsu': 0,
  'offloaded_cycles_cpu': 250000000.0,
  'offloaded_cycles_gpu': 0.0,
  'offloaded_bits': 250000.0,
  'T_uav_trans': 0.00037457354005617253,
  'T_uav_comp': 0.05,
  'T_uav_output': 3.745735400561725e-05,
  'E_comp': 0.625,
  'E_hover': 11.090646796693592,
  'E_fly': np.float64(0.0),
  'E_total': np.float64(11.715646796693592)},
 'rsu_stats': {'T_comp_rsu': {0: 0.03125, 1: 0.04375, 2: 0.25, 3: 0.2125},
  'T_queue_rsu': {0: 0.0, 1: 1000000.0, 2: 1000000.0, 3: 1000000.0},
  'T_output_rsu': {0: 0.00125, 1: 0.0025, 2: 0.01, 3: 0.01}},
 'uav_energy': {'E_batt': np.float64(499988.2843532033), 'harvested': 0.0},
 'overloaded_rsus': [0, 1, 2, 3]}